In [19]:
import pandas as pd
import numpy as np
from statistics import stdev

## Reading file with Praat results

In [20]:
df = pd.read_excel('betterresults.xlsx')

In [21]:
df

,speaker,text,try,phrase,words,syllables,vowelIntervalNum,wordIntervalNum,duration,f0,...,f0min,f1,f2,f3,f1bw,f2bw,f3bw,intensity,intensity_max,tilt
0,10,1,2,1,ты,ты,1,1,0.162558,"c(-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...",...,99.663131,"c(887.990271993513,719.588546442435,694.714891...","c(1949.902839289407,2039.4378972075558,2171.36...","c(3217.6487642231286,3325.49511144771,3465.121...","c(887.990271993513,719.588546442435,694.714891...","c(1949.902839289407,2039.4378972075558,2171.36...","c(3217.6487642231286,3325.49511144771,3465.121...",64.625157,69.170226,-128.906250
1,10,1,2,1,помог,па,1,2,0.096549,"c(-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,102.9495386...",...,94.685674,"c(1084.3898721077885,1725.1989278904505,1016.3...","c(2474.1490260422306,2907.8188562948035,2250.7...","c(3719.657871074747,4301.406426165323,3514.644...","c(1084.3898721077885,1725.1989278904505,1016.3...","c(2474.1490260422306,2907.8188562948035,2250.7...","c(3719.657871074747,4301.406426165323,3514.644...",69.798802,71.785824,-99.609375
2,10,1,2,1,помог,мок,1,3,0.182262,"c(96.76863236175305,95.79430459066666,94.94949...",...,89.200689,"c(548.9563668177481,538.2739583141623,652.2056...","c(926.383738967245,922.888749063728,1326.03152...","c(2128.9063531481324,2118.7450796320254,2857.0...","c(548.9563668177481,538.2739583141623,652.2056...","c(926.383738967245,922.888749063728,1326.03152...","c(2128.9063531481324,2118.7450796320254,2857.0...",71.626718,75.345600,-99.609375
3,10,1,2,1,своему,свъ,1,4,0.174380,"c(-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...",...,120.281481,"c(1058.9963047243184,1106.8869346918705,1094.5...","c(1133.558267901124,1182.9871590934424,1346.17...","c(2859.777440205722,2756.272703727802,2804.529...","c(1058.9963047243184,1106.8869346918705,1094.5...","c(1133.558267901124,1182.9871590934424,1346.17...","c(2859.777440205722,2756.272703727802,2804.529...",63.775657,68.949483,-131.835938
4,10,1,2,1,своему,му,1,5,0.153691,"c(121.63311252404219,120.80822832806666,120.43...",...,89.016564,"c(536.4742738612686,586.8269379930358,691.3865...","c(941.0124379641528,899.0140281139908,1267.393...","c(2442.055348564386,2451.0371265098916,2483.73...","c(536.4742738612686,586.8269379930358,691.3865...","c(941.0124379641528,899.0140281139908,1267.393...","c(2442.055348564386,2451.0371265098916,2483.73...",67.803516,69.281294,-105.468750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,9,1,2,2,вчера,фч'и,1,4,0.134904,"c(168.25806536172038,-1,-1,-1,-1,-1,-1,-1,-1,-...",...,-1.000000,"c(432.2460258574085,427.11100724375945,426.439...","c(928.0790295977897,1013.0201506487932,1177.14...","c(3056.3760501215024,3016.1759030899284,3048.1...","c(432.2460258574085,427.11100724375945,426.439...","c(928.0790295977897,1013.0201506487932,1177.14...","c(3056.3760501215024,3016.1759030899284,3048.1...",64.656811,69.465933,0.000000
296,9,1,2,2,вчера,ра,1,5,0.128772,"c(-1,-1,-1,-1,-1,-1,182.11490249262266,180.716...",...,138.030408,"c(1475.2021039822528,1497.0656660454595,1537.8...","c(2661.931265821916,2649.272789078904,2646.970...","c(3371.696489358796,3331.556565786559,3360.531...","c(1475.2021039822528,1497.0656660454595,1537.8...","c(2661.931265821916,2649.272789078904,2646.970...","c(3371.696489358796,3331.556565786559,3360.531...",75.833254,78.171516,-140.625000
297,9,1,2,2,квартиру,квър,1,6,0.193158,"c(138.4500779928358,138.23344500953374,138.042...",...,133.238497,"c(495.0373162085869,473.29708117817734,463.724...","c(1408.150206931063,1401.79306634274,1406.8788...","c(3290.894109707336,3278.7080763220188,3300.69...","c(495.0373162085869,473.29708117817734,463.724...","c(1408.150206931063,1401.79306634274,1406.8788...","c(3290.894109707336,3278.7080763220188,3300.69...",71.955543,76.066789,-158.203125
298,9,1,2,2,квартиру,т'и,1,7,0.183960,"c(168.35572526342767,172.63308264139695,175.58...",...,171.888328,"c(584.6145803344306,496.32666504355683,402.082...","c(2008.210796673186,1876.8244395448082,1731.75...","

In [22]:
def extract_f0(vector):
    """
    Transforming f0 data from c(x, x, x) to enumerated list
    """
    vector = vector[2:-1]
    vector = list(map(float, vector.split(',')))
    start_minus = 0
    for i in range(len(vector)):
        if vector[i] == -1:
            start_minus += 1
        else:
            break
    vector = vector[start_minus:]
    for i in range(len(vector), 1, -1):
        if vector[i - 1] == -1:
            vector.pop(i - 1)
        else:
            break
    return list(enumerate(vector))

In [23]:
def split_lis_by_element(lst, elem, number):
    '''
    Splitting list by several same elemens in row
    '''
    result = []
    indices = []
    in_row = 0
    previous = ""
    start = 0
    for i, element in lst:
        if previous == element:
            in_row += 1
        else:
            if in_row > number:
                indices.append((start, i))
            in_row = 1
            start = i
            previous = element
    start = 0
    for i in range(len(indices)):
        result.append(lst[start:indices[i][0]])
        start = indices[i][1]
    result.append(lst[start:])
    return result

In [24]:
def clean_list(lst):
    '''
    Removing -1
    '''
    new_list = []
    for elem in lst:
        if elem != -1:
            new_list.append(elem)
    return new_list

In [25]:
def detect_pitch_change(f_zeros, std):
    '''
    Detecting pitch change depending on std
    '''
    verdict = ''
    f_zeros = split_lis_by_element(f_zeros, -1, 5)
    for f_zero in f_zeros:
        if not f_zero:
            continue
        for i in range(len(f_zero), 1, -1):
            if f_zero[i - 1][1] == -1:
                f_zero.pop(i - 1)
        min_f = min(f_zero, key=lambda x: x[1])
        max_f = max(f_zero, key=lambda x: x[1])
        if max_f[1] - min_f[1] > 1.2 * std:
            if min_f[0] < max_f[0]:
                verdict += 'H'
            else:
                verdict += 'L'
    
    if verdict == '':
        return '-'
    else:
        return verdict

In [26]:
def mapping(syll):
    '''
    Mapping syllables on standartized variants due to the phonetic variation
    '''
    maps = {
         'а': "ра",
         'бра': "бра",
         'е': "е",
         'квар': "квар",
         'квър': "квар",
         'кто': "кто",
         'мок': "мок",
         'му': "му",
         'мъ': "му",
         'па': "па",
         'ра': "ра",
         'раw': "рал",
         'рал': "рал",
         'ру': "ру",
         'ръ': "ру",
         'ръл': "рал",
         "св'и": "свэ",
         'свъ': "свэ",
         'свэ': "свэ",
         'съ': "свэ",
         "т'и": "т'и",
         "т'ир": "т'и",
         'ту': "ту",
         'тъ': "ту",
         'ты': "ты",
         'уб': "уб",
         'убрал': "рал",
         "фч'е": "фч'и",
         "фч'и": "фч'и",
         "фч'ь": "фч'и",
         "фш'е": "фч'и",
         "ч'и": "фч'и",
         "ч'ь": "фч'и"
    }
    if syll in maps:
        return maps[syll]
    else:
        return syll

# Основной тон

In [27]:
all_contours = []
for x in pd.DataFrame(df.groupby(['speaker', 'phrase']))[1]:
    pitch_contour = []
    x.f0 = x.f0.apply(extract_f0)
    speaker = x.speaker.iloc[0,]
    text = x.text.iloc[0,]
    phrase = x.phrase.iloc[0,]
    std = stdev(clean_list(z[1] for z in sum(x.f0, [])))
    for y in x.iterrows():
        pitch_contour.append(
            {
                'speaker': speaker,
                'text': text,
                'phrase': phrase,
                'word': y[1].words,
                'syllable': mapping(y[1].syllables),
                'contour': detect_pitch_change(y[1].f0, std)
            }
        )
    all_contours.append(pitch_contour)

In [28]:
final = pd.DataFrame(all_contours[0])[['text', 'phrase', 'word', 'syllable', 'contour']]
for contour in all_contours[1:]:
    to_merge = pd.DataFrame(contour)[['text', 'phrase', 'word', 'syllable', 'contour']]
    final = final.merge(to_merge, on=['text', 'phrase', 'word', 'syllable'], how='outer')

C:\Users\anton\Anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'contour_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


In [29]:
final = pd.DataFrame(list(final.apply(lambda x : (x[x.notnull()].values.tolist()+x[x.isnull()].values.tolist()), 1))).dropna(axis=1, how='all')

In [30]:
final.to_excel('analyze_pitch.xlsx')

# Длительность

In [31]:
all_durs = []
for x in pd.DataFrame(df.groupby(['speaker', 'phrase']))[1]:
    duration = []
    speaker = x.speaker.iloc[0,]
    text = x.text.iloc[0,]
    phrase = x.phrase.iloc[0,]
    base_dur = x.duration.iloc[2,]
    x.duration = x.duration.apply(lambda x: x / base_dur * 100)
    for y in x.iterrows():
        duration.append(
            {
                'speaker': speaker,
                'text': text,
                'phrase': phrase,
                'word': y[1].words,
                'syllable': mapping(y[1].syllables),
                'duration': y[1].duration
            }
        )
    all_durs.append(duration)

In [32]:
final_dur = pd.DataFrame(all_durs[0])[['text', 'phrase', 'word', 'syllable', 'duration']]
for dur in all_durs[1:]:
    to_merge = pd.DataFrame(dur)[['text', 'phrase', 'word', 'syllable', 'duration']]
    final_dur = final_dur.merge(to_merge, on=['text', 'phrase', 'word', 'syllable'], how='outer')

C:\Users\anton\Anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'duration_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


In [33]:
final_dur = pd.DataFrame(list(final_dur.apply(lambda x : (x[x.notnull()].values.tolist()+x[x.isnull()].values.tolist()), 1))).dropna(axis=1, how='all')

In [34]:
final_dur.to_excel('analyze_dur.xlsx')